In [2]:



import fasttext
#model = fasttext.load_model('practical4/fasttext/wiki.en.bin')
#print model.words # list of words in dictionary

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import log_loss, precision_score, recall_score, f1_score
%matplotlib inline

df_orig = pd.read_csv('practical4/quora-train.csv')
df_orig.head(5)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


df_orig.question2.replace(np.nan, 'nan', inplace=True)
all_q = list(df_orig.question1) + list(df_orig.question2)
tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(all_q)


splitter = StratifiedShuffleSplit(
    n_splits=1,
    test_size=0.2,
    random_state=RANDOM_STATE,
)

ix_train, ix_test = next(splitter.split(df_orig, df_orig.is_duplicate))

X = np.zeros((len(df_orig), 1))
y = df_orig.is_duplicate.values

In [27]:
import re
def norm_sentence(norm_text):
    for char in ['.', '"', ',', '(', ')', '!', '?', ';', ':', '_']:
        norm_text = norm_text.replace(char, ' ')
    return norm_text;


In [ ]:
vocabs = dict()
f = open('practical4/quora.vocab', 'w')
for s in all_q:
    sentence = re.split('[^a-z\\d]',norm_sentence(s).lower())
    if '' in sentence:
        sentence.remove('')
    for word in sentence:
        if word not in vocabs:
            vocabs[word] = 0
            f.write(word+'\n')
f.close()

In [ ]:
#word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True)
from gensim.models.wrappers.fasttext import FastText
embedding_model = FastText.load_word2vec_format('practical4/quora.vec')

#return embedding_model.wmdistance(doc1, doc2)

In [29]:
def evaluate(y_true, y_pred, threshold=0.5):
    y_pred_label = y_pred >= threshold
    
    print('Log loss: ', log_loss(y_true, y_pred))
    print('Precision:', precision_score(y_true, y_pred_label))
    print('Recall:   ', recall_score(y_true, y_pred_label))
    print('F1 score: ', f1_score(y_true, y_pred_label))

In [28]:
import nltk
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
import re
stopwords = nltk.corpus.stopwords.words('english')

q1 = list(df_orig.question1)
q2 = list(df_orig.question2)
    


train_arrays = []
train_labels = []
test_arrays = []
test_labels = []
for i in range(len(q1)):
    s_q1 = q1[i].lower();
    s_q2 = q2[i].lower()
    sentence_q1 = re.split('[^a-z\\d]',norm_sentence(s_q1))
    sentence_q2 = re.split('[^a-z\\d]',norm_sentence(s_q2))
    
    sentence_q1 = [w for w in sentence_q1 if w not in stopwords]
    sentence_q2 = [w for w in sentence_q2 if w not in stopwords]
    
    while '' in sentence_q1:
        sentence_q1.remove('')
    while '' in sentence_q2:
        sentence_q2.remove('')
   
 
    if df_orig.id[i] in ix_train :
        train_arrays.append(embedding_model.wmdistance(sentence_q1, sentence_q2))
        train_labels.append(df_orig.is_duplicate[i])
    else:
        test_arrays.append(embedding_model.wmdistance(sentence_q1, sentence_q2))
        test_labels.append(df_orig.is_duplicate[i])

   
   

/Users/maxgirilishen/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [32]:
train_arrays = np.nan_to_num(train_arrays)
train_arrays[np.isneginf(train_arrays)] = 1
train_arrays[np.isinf(train_arrays)] = 1
test_arrays = np.nan_to_num(test_arrays)
test_arrays[np.isneginf(test_arrays)] = 1
test_arrays[np.isinf(test_arrays)] = 1
classifier.fit(train_arrays.reshape(-1,1), train_labels)
print(classifier.score(test_arrays.reshape(-1,1), test_labels))
y_pred_test = classifier.predict(test_arrays.reshape(-1,1))
evaluate(test_labels, y_pred_test)

0.63079720003
('Log loss: ', 12.751812952549763)
('Precision:', 0.0)
('Recall:   ', 0.0)
('F1 score: ', 0.0)
